In [ ]:
%run default-imports.ipynb

In [ ]:
rfe_experiments = unpickle('./experiments/experiments_rfe_sinai.d') or {}


In [ ]:
len(rfe_experiments)

In [ ]:
algorithm = algorithms.RF
metric = 'auc'

experiments = [exp for exp in rfe_experiments.values() if exp['parameters']['rfe_mode'] == 'prefit' and exp['parameters']['algorithm'] == algorithm ]

#experiments = [exp['parameters'].get('feature_rank') for exp in rfe_experiments.values() if exp['parameters']['rfe_mode'] == 'prefit']
#print(experiments[0].keys())
[print(f"({f[0]},{f[1]}, {f[2]})") for f in [(k, v[metric]['mean'], v[metric]['ci']) for k,v in experiments[0]['performance']['rfe'].items()]]


In [ ]:
data_points = {}

#experiment = [exp for exp in rfe_experiments.values() if exp['parameters']['rfe_mode'] == 'feature_importance' and exp['parameters']['algorithm'] == algorithm ][0]
#data_points['Feature Importance'] = [(k, v[metric]['mean'], v[metric]['ci']) for k,v in experiment['performance']['rfe'].items()]

#experiment = [exp for exp in rfe_experiments.values() if exp['parameters']['rfe_mode'] == 'weighted_explanations' and exp['parameters']['algorithm'] == algorithm ][0]
#data_points['Weighted Explanations'] = [(k, v[metric]['mean'], v[metric]['ci']) for k,v in experiment['performance']['rfe'].items()]

experiment = [exp for exp in rfe_experiments.values() if exp['parameters']['rfe_mode'] == 'prefit' and exp['parameters']['algorithm'] == algorithm ][0]
data_points['SINAI w/ MIMIC'] = [(k, v[metric]['mean'], v[metric]['ci']) for k,v in experiment['performance']['rfe'].items()]


rfe_experiments_mimic = unpickle('./experiments/experiments_rfe_mimic.d') or {}
experiment = [exp for exp in rfe_experiments_mimic.values() if exp['parameters']['rfe_mode'] == 'weighted_explanations' and exp['parameters']['algorithm'] == algorithm ][0]
data_points['MIMIC'] = [(k, v[metric]['mean'], v[metric]['ci']) for k,v in experiment['performance']['rfe'].items()]




In [ ]:
%matplotlib notebook

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.axes()
ax.set_xlabel('Number of Features')
ax.set_ylabel('Area Under the ROC Curve')
ax.set_title('Explanation-Driven Recursive Feature Elimination')

for rfe_mode in data_points:
    plot_data = sorted(data_points[rfe_mode], key=lambda x: int(x[0]), reverse=False)
    X = np.array([int(d[0]) for d in plot_data])
    y = np.array([d[1] for d in plot_data])
    ci = np.array([d[2] for d in plot_data])
    line, = ax.plot(X,y,'-')
    line.set_label(rfe_mode)
    print(f"Top number of features ({rfe_mode}): {X[y.argmax()]}, Metric={y.max()}")
    ax.axvline(X[y.argmax()], linestyle='-.', linewidth=1)
    #ax.errorbar(X, y, yerr=ci/2, color='gray', alpha=0.5, capsize=3)
    ax.fill_between(X, y+ci/2, y-ci/2, alpha=0.2, edgecolor='gray')
    #ax.fill_between(X, y+ci/2, y-ci/2, alpha=0.2)

mimic = sorted(data_points['MIMIC'], key=lambda x: int(x[0]), reverse=False)
sinai = sorted(data_points['SINAI w/ MIMIC'], key=lambda x: int(x[0]), reverse=False)
X = np.array([int(d[0]) for d in mimic])
y = np.array([d[1] for d in mimic]) - np.array([d[1] for d in sinai])[:90]

#line_feat, = ax.plot(X,y,'-',color='red')
#line_feat.set_label('Difference')
ax.set_xlim((10,100))
ax.legend()
plt.tight_layout()


In [ ]:
plt.savefig('test.pdf')

In [ ]:
from scipy import stats

y1 = y_feat
y2 = y_exp
y1 = y1[:-1]

plt.clf()
ax = plt.axes()
ax.hist(y1-y2)
plt.show()

from statsmodels.graphics.gofplots import qqplot
qqplot(y1-y2, line='s')

In [ ]:
print(stats.ttest_rel(y1,y2))
print(f"Mean of feat: {y1.mean()}")
print(f"Mean of exp: {y2.mean()}")
print(f": {y1.mean() - y2.mean()}")




In [ ]:
np.arange(0, 1.01, step=0.01)